In [1]:
import pandas as pd
import json
import glob, os
import requests
from datetime import datetime
import hashlib

%load_ext autoreload
%autoreload 2

from dotenv import load_dotenv
load_dotenv('.env')

URL = os.getenv("KG_URL_FCT")
TOK3N = os.getenv("KG_TOKEN")

In [2]:
import importlib.util
import sys
spec = importlib.util.spec_from_file_location("llm", "../../../utils/llm.py")
h = importlib.util.module_from_spec(spec)
sys.modules["llm"] = h
spec.loader.exec_module(h)

In [3]:
def ask(CONTEXT,QUESTION,prefix,functions=h.functionsT35,overwrite=False,MODEL="gpt-3.5-turbo-1106"):
    REQ = {
        "context": CONTEXT,
        "question": QUESTION,
        "model": MODEL,
        "functions": functions,
        "token": TOK3N,
        "overwrite": overwrite,
        "source": "local-risksT3.5"
    }

    H = h.hashme(CONTEXT+QUESTION)
    cached = "cache/"+prefix+"_"+H+".json"
    if not os.path.isfile(cached) or overwrite:
        x = requests.post(URL+"fct/", json = REQ)
        answer = json.loads(x.text)["messages"][-1]
        h.svt(cached,answer)
    else:
        answer = h.ldt(cached)
    
    return answer

In [1]:
from langchain.vectorstores import Chroma
vectordb = Chroma(persist_directory="../../../bok/data/vectorDB/")

DOCS  = list(vectordb.get()["documents"])
SRC = [x["article"] for x in vectordb.get()["metadatas"]]
print(len(DOCS),len(SRC))


2619 2619


In [5]:
RISKS = []
for k in range(len(SRC)):
    done = False
    while not done:
        try:
            answer = ask("Do a risk assessment of the following text - list as many risks as possible, ideally at least 3.",DOCS[k],prefix="risk",functions=h.functionsT35,overwrite=True)
            D = pd.DataFrame(json.loads(answer["function_call"]["arguments"])["risks"])
            D["src"] = SRC[k]
            D["hash"] = h.hashme(DOCS[k])
            if not len(D.columns) == 8:
                print("Must redo")
                answer = ask("Do a risk assessment of the following text - list as many risks as possible, ideally at least 3.",DOCS[k],prefix="risk",functions=h.functionsT35,overwrite=True)
                done = False
            else:
                done = True
        except:
            done = False
    RISKS.append(D)
answer

In [ ]:
pd.concat(RISKS).reset_index(drop=True)

,Description,Impact,Mitigation,Technologies,People,Owner,src,hash
0,Spread of zoonotic infections from wild animal...,Potential for widespread illness and mortality...,Enhanced surveillance of animal markets and ha...,"Surveillance technology, genetic sequencing fo...","General public, healthcare workers, and indivi...","Health organizations, governments, and wildlif...",abideen_mitigation_2020,f5f5af1eef7fe66226065027d070a105
1,Inadequate preparedness and response to pandemics,"Severe economic disruption, increased morbidit...","Development of pandemic response plans, invest...","Healthcare management systems, telemedicine, a...","Government officials, healthcare workers, and ...","Government agencies, healthcare organizations,...",abideen_mitigation_2020,f5f5af1eef7fe66226065027d070a105
2,High mortality rate of past pandemics (1918 in...,"Mass casualties, strain on healthcare systems,...","Global coordination, rapid response plans, inv...","Vaccines, advanced medical treatments, early d...","Global population, healthcare workers, governm...","Global health organizations, national governments",abideen_mitigation_2020,e2fefe64d3f6f552fd8c225341c476cb
3,Ease of transmission of influenza viruses,"Global pandemic, strain on healthcare systems,...","Vaccination campaigns, public health education...","Vaccines, public health surveillance systems, ...","Global population, healthcare workers, travelers","Public health agencies, healthcare organizations",abideen_mitigation_2020,e2fefe64d3f6f552fd8c225341c476cb
4,Zoonotic transmission of viruses from animals ...,"Spread of new diseases, morbidity and mortalit...","Wildlife trade regulation, surveillance of ani...","Genomic sequencing, animal disease surveillanc...","Wildlife traders, animal handlers, healthcare ...","Environmental agencies, public health organiza...",abideen_mitigation_2020,e2fefe64d3f6f552fd8c225341c476cb
5,Reporting sensitive information (number of cas...,"Causing panic, misinformation, or data privacy...","Provide context, verify the information, and e...","Data encryption, secure data storage, fact-che...","General public, government officials, healthca...","Information analyst, public health authorities",abideen_mitigation_2020,42072a8f15cf533494857fe51458bb2c
6,COVID-19 transmission risk,"Public health crisis, overwhelmed healthcare s...","Mass testing, contact tracing, social distanci...","Testing kits, contact tracing apps, data analy...","General public, healthcare workers, public hea...","Health organizations, government, research ins...",abideen_mitigation_2020,42072a8f15cf533494857fe51458bb2c
7,Strategic approach to pandemic mitigation,"Ineffective response, prolonged crisis, econom...","Data-driven decision-making, coordinated inter...","Big data analysis, predictive modeling, commun...","Global leaders, policymakers, healthcare workers","International organizations, government agenci...",abideen_mitigation_2020,42072a8f15cf533494857fe51458bb2c
8,Countries with lower financial indexes facing ...,Risk of ineffective pandemic management and in...,"Financial aid and support, capacity building, ...","Remote monitoring systems, data analytics for ...","Countries with lower financial indexes, vulner...","International organizations, donor countries, ...",abideen_mitigation_2020,60882b1068cfae1241b7235cc557b99a
9,Scarcity of medical equipment and medication l...,"Increased mortality, inadequate patient care, ...","Global supply chain management, strategic stoc...","Supply chain management software, inventory tr...","Healthcare workers, patients, and vulnerable p...","Healthcare organizations, governments, and int...",abideen_mitigation_2020,60882b1068cfae1241b7235cc557b99a
